In [1]:
import numpy as np
from numpy.linalg import norm, inv
import pandas as pd

In [2]:
# define the first function of interest
def f(x):
    return (5 * x[0] - x[1])**4 + (x[0] - 2)**2 + x[0] - 2*x[1] + 12

# define the gradient of the said function
def gradient(x):
    dx1 = 20 * ( 5 * x[0] - x[1])**3 + 2 * (x[0] - 2) + 1
    dx2 = -4 * ( 5 * x[0] - x[1])**3 - 2
    return np.array([dx1, dx2])

# define Rosenbrock’s curved valley function.
def rosenbrock(x):
    return 100*np.power(x[1]-np.power(x[0],2),2) + np.power(1-x[0],2)

# define its gradient
def gradient_rosenbrok(x):
    dx1 = 400*(np.power(x[0],3) - x[0]*x[1]) + 2 * (x[0]-1)
    dx2 = 200*(x[1]-np.power(x[0],2))
    return np.array([dx1, dx2])

# to be used for line search
# parameter set  (ϵ2, a, b) = (0.005,-100, 100)
def Golden_Section(f, a=-100, b=100, epsilon=0.005):
    gamma = (1 + np.sqrt(5))/2
    c = 1/gamma

    x = b - c*(b-a) # defining the x0
    y = a + c*(b-a) # defining the y0

    fx = f(x)       # defining the f(x0)
    fy = f(y)       # defining the f(y0)

    while abs(b-a) > epsilon:
        if fx > fy:
            a = x
            x = y
            fx = fy

            y = a + c*(b-a)
            fy = f(y)

        else:
            b = y
            y = x
            fy = fx

            x = b - c*(b-a)
            fx = f(x)

    return x

In [3]:
def BFGS(function, gradient, x0, epsilon=1e-6):

    # lists to keep track of the variables
    x_list, f_list, d_list, alpha_list, xnew_list = [], [], [], [], []

    f, grad_f = function, gradient

    n = len(x0)
    x = np.array(x0, dtype=float)
    H = np.eye(n)  # H0 = I
    g = grad_f(x)

    # do until || ∇f(x) || <= ϵ1, i.e. until gradient diminishes
    while norm(g) > epsilon:
        g = grad_f(x) # gradient at the current point
        x_list.append(x)
        f_list.append(f(x))

        # Search direction
        d = -H.dot(g)
        d_list.append(d)

        # Line search

        def h(a): return f(x + a * d)  # h(α) = f(x + α*d),
        alpha = Golden_Section(h) # α = argmin h(α)
        alpha_list.append(alpha)

        # Update position
        x_new = x + alpha * d
        xnew_list.append(x_new)

        # Compute difference vectors
        sk = x_new - x
        yk = grad_f(x_new) - g

        # H(k+1)
        H = H + (sk.reshape(1, 2) @ yk.reshape(2, 1) + yk.reshape(1, 2) @ H @ yk.reshape(2, 1)) * sk.reshape(2, 1) @ sk.reshape(1, 2) / (sk.reshape(1, 2) @ yk.reshape(2, 1))**2 \
              - (H @ yk.reshape(2, 1) @ sk.reshape(1, 2) + sk.reshape(2, 1) @ yk.reshape(1, 2) @ H) / (sk.reshape(1, 2) @ yk.reshape(2, 1))

        # update x
        x = x_new

    return pd.DataFrame({'x(k)': x_list, 'f(x(k))': f_list, 'd(k)': d_list, 'alpha(k)': alpha_list, 'x(k+1)': xnew_list}).rename_axis('k')

$
\text{  
}
$

$
\text{  
}
$

$$
(\epsilon_1, x^{(0)}) = (1e-6, (0, 0))
$$

In [4]:
x0 = np.array([0.0, 0.0])
BFGS(f, gradient, x0)

,x(k),f(x(k)),d(k),alpha(k),x(k+1)
k,,,,,
0,"[0.0, 0.0]",16.000000,"[3.0, 2.0]",0.046871,"[0.14061287713497048, 0.09374191808998032]"
1,"[0.14061287713497048, 0.09374191808998032]",15.548294,"[0.9166458414032581, 4.8477695072739975]",6.432373,"[6.036821235065613, 31.276405707141734]"
2,"[6.036821235065613, 31.276405707141734]",-26.796533,"[1.0358127262612402, 3.1816373004793435]",0.170177,"[6.213092539551369, 31.817846590360414]"
3,"[6.213092539551369, 31.817846590360414]",-27.352004,"[0.3134900823680373, 1.598826241059889]",0.929757,"[6.5045621472823925, 33.30436652678602]"
4,"[6.5045621472823925, 33.30436652678602]",-27.439978,"[-0.0031800511171972386, -0.007005833338520147]",1.367711,"[6.500212757928666, 33.29478457485708]"
5,"[6.500212757928666, 33.29478457485708]",-27.440551,"[-0.00021314280007115943, -0.00108632337326606]",0.997056,"[6.500000242586265, 33.29370144944281]"
6,"[6.500000242586265, 33.29370144944281]",-27.440551,"[-2.4423554465826647e-07, -9.271999171958773e-07]",0.927828,"[6.500000015977674, 33.293700589160714]"
7,"[6.500000015977674, 33.293700589160714]",-27.440551,"[-1.5978903994107714e-08, -6.318302527153341e-08]",-1.317491,"[6.500000037029732, 33.293700672403766]"


$$
(\epsilon_1, x^{(0)}) = (5e-8, (3, 3))
$$

In [ ]:
x0 = np.array([3.0, 3.0])
BFGS(f, gradient, x0, epsilon=5e-8)

,x(k),f(x(k)),d(k),alpha(k),x(k+1)
k,,,,,
0,"[1.0, -1.0]",1312.000000,"[-4319.0, 866.0]",0.000228,"[0.016630085298312114, -0.8028251108748179]"
1,"[0.016630085298312114, -0.8028251108748179]",18.172187,"[0.4960909314403537, 2.495176592367404]",13.167874,"[6.54909316204318, 32.053346866052145]"
2,"[6.54909316204318, 32.053346866052145]",-24.633884,"[-0.001968431558078605, 0.010163553365953008]",74.143061,"[6.403147620893014, 32.8069038235775]"
3,"[6.403147620893014, 32.8069038235775]",-27.431146,"[0.09626197328929821, 0.4898248146506808]",0.985764,"[6.498039184262831, 33.28975536766382]"
4,"[6.498039184262831, 33.28975536766382]",-27.440417,"[0.006568746726958329, 0.013241761896103169]",0.298533,"[6.500000170493812, 33.293708467673845]"
5,"[6.500000170493812, 33.293708467673845]",-27.440551,"[-1.7071911250871755e-07, -7.89076192240877e-06]",1.005227,"[6.499999998882277, 33.2937005356635]"
6,"[6.499999998882277, 33.2937005356635]",-27.440551,"[1.117853082317319e-09, -9.678613836937007e-09]",1.114120,"[6.5000000001277, 33.293700524880364]"
7,"[6.5000000001277, 33.293700524880364]",-27.440551,"[-1.2769944154512703e-10, 1.1037383123363326e-09]",-4.182741,"[6.500000000661833, 33.29370052026371]"
8,"[6.500000000661833, 33.29370052026371]",-27.440551,"[-6.618330328150199e-10, 5.720391339620866e-09]",-0.992743,"[6.500000001318863, 33.293700514584835]"


In [ ]:
BFGS(f, gradient, x0).to_latex()

'\\begin{tabular}{llrlrl}\n\\toprule\n & x(k) & f(x(k)) & d(k) & alpha(k) & x(k+1) \\\\\n\\midrule\n0 & [3. 3.] & 20746.000000 & [-34563.   6914.] & 0.000228 & [-4.86946385  4.57421153] \\\\\n1 & [-4.86946385  4.57421153] & 699702.041862 & [7.65437094 0.07869799] & 0.766787 & [0.9998083  4.63455612] \\\\\n2 & [0.9998083  4.63455612] & 4.748729 & [0.42308332 2.11546463] & 12.999854 & [ 6.49982982 32.13528796] \\\\\n3 & [ 6.49982982 32.13528796] & -25.504749 & [0.00029455 0.00213639] & 99.998071 & [ 6.52928381 32.34892247] \\\\\n4 & [ 6.52928381 32.34892247] & -25.646316 & [0.10170612 2.0192909 ] & 0.720144 & [ 6.60252684 33.80310215] \\\\\n5 & [ 6.60252684 33.80310215] & -27.430000 & [-0.08290845 -0.40319943] & 1.172055 & [ 6.50535354 33.33053007] \\\\\n6 & [ 6.50535354 33.33053007] & -27.440136 & [-0.02004171 -0.13847177] & 0.262727 & [ 6.50008805 33.29414986] \\\\\n7 & [ 6.50008805 33.29414986] & -27.440551 & [-8.80707223e-05 -4.49323714e-04] & 1.000177 & [ 6.49999996 33.29370046] \\\

In [ ]:
x0 = np.array([0.0, 0.0])
BFGS(rosenbrock, gradient_rosenbrok, x0)

,x(k),f(x(k)),d(k),alpha(k),x(k+1)
k,,,,,
0,"[0.0, 0.0]",1.000000e+00,"[2.0, -0.0]",0.079556,"[0.15911207650349832, 0.0]"
1,"[0.15911207650349832, 0.0]",7.711858e-01,"[13.778345741464806, 5.2483391581757095]",0.008399,"[0.2748357745228913, 0.04408056142881427]"
2,"[0.2748357745228913, 0.04408056142881427]",6.247995e-01,"[0.08800153169897451, 0.11233570506368991]",0.728315,"[0.33892860990782725, 0.1258963402396471]"
3,"[0.33892860990782725, 0.1258963402396471]",4.491677e-01,"[0.18334994506835808, 0.10413207746407221]",0.641552,"[0.4565571664169125, 0.19270250128771463]"
4,"[0.4565571664169125, 0.19270250128771463]",3.201110e-01,"[0.05760626797744891, 0.04954785591636829]",1.221083,"[0.526899202585973, 0.2532045472514102]"
5,"[0.526899202585973, 0.2532045472514102]",2.834493e-01,"[0.05917508747489958, 0.0822483046847078]",4.726921,"[0.8066151856845215, 0.6419858128951756]"
6,"[0.8066151856845215, 0.6419858128951756]",4.486653e-02,"[-0.04514902662279005, -0.025667244047023082]",22.690490,"[-0.21783834226568566, 0.0595834737169757]"
7,"[-0.21783834226568566, 0.0595834737169757]",1.497844e+00,"[-0.5189425737889548, -0.47210497346803454]",-2.609503,"[1.1363439400463595, 1.2915428903321362]"
8,"[1.1363439400463595, 1.2915428903321362]",1.859671e-02,"[0.20833732506592548, 0.24874353407832522]",-5.139906,"[0.06550974871458015, 0.013024597561095863]"


In [ ]:
x0 = np.array([1.0, -1.0])
BFGS(rosenbrock, gradient_rosenbrok, x0, epsilon=1e-10)

,x(k),f(x(k)),d(k),alpha(k),x(k+1)
k,,,,,
0,"[1.0, -1.0]",4.000000e+02,"[-800.0, 400.0]",0.000228,"[0.8178522964201551, -0.9089261482100776]"
1,"[0.8178522964201551, -0.9089261482100776]",2.489812e+02,"[45.31969733658014, 153.9880033983467]",0.018499,"[1.6562304127716536, 1.9397286270200707]"
2,"[1.6562304127716536, 1.9397286270200707]",6.497106e+01,"[-0.36404252487431943, 0.10881958348619784]",0.654774,"[1.4178649891996047, 2.010980814163802]"
3,"[1.4178649891996047, 2.010980814163802]",1.746521e-01,"[-0.0016264046097769957, -0.004747240479249991]",51.486221,"[1.334127562346528, 1.7665633426468792]"
4,"[1.334127562346528, 1.7665633426468792]",1.294181e-01,"[-0.039624550514448374, -0.10365763488944209]",2.396032,"[1.239185875246302, 1.5181963426569516]"
5,"[1.239185875246302, 1.5181963426569516]",8.743472e-02,"[-0.03164022238093351, -0.051217597248677395]",26.224700,"[0.409430539897791, 0.1750302274140083]"
6,"[0.409430539897791, 0.1750302274140083]",3.542436e-01,"[-0.24839480179185655, -0.5753792229897549]",-6.032892,"[1.9079695404156944, 3.646230910833876]"
7,"[1.9079695404156944, 3.646230910833876]",8.278698e-01,"[0.12024557956594926, 0.1432549477303976]",-21.824683,"[-0.7163521241377495, 0.5197370784584106]"
8,"[-0.7163521241377495, 0.5197370784584106]",2.950190e+00,"[-0.22704175677664098, 2.1989092056512436]",-0.004822,"[-0.7152572293787886, 0.5091329755000853]"
